In [1]:
# import libraries
import pandas as pd
import numpy as np
import random

In [2]:
# Create competitor list
name_list = list(["Team1", "Team2", "Team3"]) # , etc... 

## Note: this is the main input that is needed and the other columns are built around this input. 

In [4]:
# How many games until all competitors play one another x times? 
def max_game_calc(competitor_list, T):
    N = len(competitor_list) # # of teams/competitors
    # T = 1 # of times each team/competitor play one another
    NT_output = (N*T)*(N-1)/2
    return(NT_output)

# max_game_calc(name_list, 1) # run function 

In [5]:
# Build data set

## create empty values to be update 
vs_count = list(np.repeat(0,len(name_list))) # count of how many times a list item has faced-off with another list item
wins = list(np.repeat(0,len(name_list))) # total wins
losses = list(np.repeat(0,len(name_list))) # total losses
ties = list(np.repeat(0,len(name_list))) # total ties
win_pct = list(np.repeat("NA",len(name_list))) # total ties
elo_score = list(np.repeat(1000,len(name_list))) # starting score is 1000 by default
vs_history = list(np.repeat("",len(name_list))) # match up history
initial_order_number = random.sample(range(len(name_list)), len(name_list)) # random array

## create series data for artist list
data_handoff = {'initial_order_number':initial_order_number,
                'name': name_list,
                'elo_score': elo_score,
                'vs_count': vs_count,
                'wins': wins,
                'losses': losses,
                'ties': ties,
                'win_pct':win_pct,
                'vs_history': vs_history                
               }

## create dataset
global data
data = pd.DataFrame.from_dict(data_handoff)

## view data
data

,initial_order_number,name,elo_score,vs_count,wins,losses,ties,win_pct,vs_history
0,3,1,1000,0,0,0,0,NA,
1,4,2,1000,0,0,0,0,NA,
2,2,3,1000,0,0,0,0,NA,
3,1,4,1000,0,0,0,0,NA,
4,0,5,1000,0,0,0,0,NA,


In [6]:
# search function
def get_data(input):
    out = data.loc[data['name'] == input]
    return out

In [7]:
# Sort data for selection
def reorder_data():
    global data
    if(data['vs_count'].sum()==0): # if cumulative count is zero,
        data = data.sort_values('initial_order_number', ascending=True) # then order by initial randomization, 
    else: ### else order by match count and then initial randomizaion 
        data = data.sort_values(by = ['vs_count', 'win_pct', 'initial_order_number'], ascending = [True, False, True],na_position = 'first')

reorder_data()


In [8]:
# Sort data for selection
def ranked_data(data_input):
    if(data['vs_count'].sum()==0): # if cumulative count is zero,
        data_output = data_input.sort_values('initial_order_number', ascending=True) # then order by initial randomization, 
    else: ### else order by match count and then initial randomizaion 
        data_output = data_input.sort_values(by = ['wins', 'win_pct'], ascending = [False, False],na_position = 'first')
    return(data_output)

In [9]:
# function that combines names to create matchup headline
def matchup_combiner(blue_corner, red_corner): # input: opponent_1, opponent_2
    matchup = [blue_corner, red_corner] # place individual opponents in array
    matchup = sorted(matchup) # sort opponents alphabetically
    matchup_headline = matchup[0] + " vs " + matchup[1] # create matchup headline 
    return matchup_headline

In [10]:
# select 
def matchup_selector(data_input):
    # set variable values for loop below
    i=0
    keep_searching="yes"
    
    while keep_searching=="yes":
        blue = data_input.iloc[0+i]
        red = data_input.iloc[1+i]
        matchup = matchup_combiner(blue['name'], red['name']) 

        # check if match up has already occurred
        in_history = data_input['vs_history'].str.contains(matchup).any()==True # skip if true (b/c already in match history) | output True, False
        if(in_history==True):
            keep_searching="yes"
        else:
            keep_searching="no"
        
        i+=1 # add one to loop

    return(blue, red, matchup) # return data for blue corner, data for red corner, and matchup text

In [11]:
def get_basic_matchup_data():
    mu_name = [current_matchup[0]['name'], current_matchup[1]['name']]
    mu_elo_score = [current_matchup[0]['elo_score'], current_matchup[1]['elo_score']]
    mu_vs_count = [current_matchup[0]['vs_count'], current_matchup[1]['vs_count']]
    mu_wins = [current_matchup[0]['wins'], current_matchup[1]['wins']]
    mu_losses = [current_matchup[0]['losses'], current_matchup[1]['losses']]
    mu_ties = [current_matchup[0]['ties'], current_matchup[1]['ties']]
    mu_win_pct = [current_matchup[0]['win_pct'], current_matchup[1]['win_pct']]
    mu_vs_history = [current_matchup[0]['vs_history'], current_matchup[1]['vs_history']]
    mu_initial_order_number  = [current_matchup[0]['initial_order_number'], current_matchup[1]['initial_order_number']]

    ## create series data for artist list
    mu_handoff = {'initial_order_number':mu_initial_order_number,
                  'name': mu_name,
                    'elo_score': mu_elo_score,
                    'vs_count': mu_vs_count,
                    'wins': mu_wins,
                    'losses': mu_losses,
                    'ties': mu_ties,
                    'win_pct': mu_win_pct,
                    'vs_history': mu_vs_history
                   }

    ## create dataset
    mu_data = pd.DataFrame.from_dict(mu_handoff)
    return(mu_data)

# get_basic_matchup_data() # run function to update matchup data 

In [12]:
def determine_outcome_from_predictions(pct1, pct2):
    if(pct1==pct2):
        out=["T", "T"] # ties
    elif(pct1>pct2):
        out=["W","L"] # win/loss
    elif(pct1<pct2):
        out=["L", "W"] # loss/win 
    return(out)

# determine_outcome_from_predictions(.50, .50) # sample input

In [13]:
# calculate elo
def elo_prediction(blue_elo, red_elo):
    
    kfactor = 32 # arbitrary number (max. amount an elo rating can change at one time)
    
    blue_transformed_rating = 10**(blue_elo/400)
    red_transformed_rating = 10**(red_elo/400)
    
    blue_exp_win_pct = blue_transformed_rating/(blue_transformed_rating+red_transformed_rating)
    red_exp_win_pct = red_transformed_rating/(blue_transformed_rating+red_transformed_rating)
    
    blue_adj_elo_if_blue_wins = blue_elo+kfactor*(1-blue_exp_win_pct)
    red_adj_elo_if_blue_wins = red_elo+kfactor*(0-red_exp_win_pct)
    
    blue_adj_elo_if_red_wins = blue_elo+kfactor*(0-blue_exp_win_pct)
    red_adj_elo_if_red_wins = red_elo+kfactor*(1-red_exp_win_pct)
    
    blue_adj_elo_if_tie = blue_elo+kfactor*(.5-blue_exp_win_pct)
    red_adj_elo_if_tie = red_elo+kfactor*(.5-red_exp_win_pct)
    
    out_corner = ["Blue", "Red"]
    out_elo = [blue_elo, red_elo]
    
    out_exp_win_pct_temp1 = [blue_exp_win_pct, red_exp_win_pct]
    out_exp_win_pct = [ '%.2f' % elem for elem in out_exp_win_pct_temp1 ] # round items in list to two decimals        
    out_prediction = determine_outcome_from_predictions(blue_exp_win_pct, red_exp_win_pct)

    out_handoff = {'corner': out_corner,
                'elo': out_elo,
                'exp_win_pct': out_exp_win_pct,
                'prediction': out_prediction,
                'Adj_Rating_if_Blue_Wins': [blue_adj_elo_if_blue_wins, red_adj_elo_if_blue_wins],
                'Adj_Rating_if_Red_Wins': [blue_adj_elo_if_red_wins, red_adj_elo_if_red_wins],
                'Adj_Rating_if_Tie': [blue_adj_elo_if_tie, red_adj_elo_if_tie],
                }
    out_data = pd.DataFrame.from_dict(out_handoff)

    return(out_data)
    
# elo_prediction(1500,2000) # example input    

In [14]:
# get user input for matchup outcome
def matchup_outcome_user_input():
    option_a = "1) " + current_matchup[0]['name']
    option_b = "2) " + current_matchup[1]['name']
    option_c = "3) Tie"
    prompt_text = current_matchup[0]['name'] + " vs " + current_matchup[1]['name'] + "\nWho wins? [input: 1, 2, 3] \n" + option_a + "\n" + option_b + "\n" + option_c + "\n"
    prompt_text_err = "\nInvalid input. Try again.\n" + current_matchup[2] + "\nWho wins? [input: 1, 2, 3] \n" + option_a + "\n" + option_b + "\n" + option_c + "\n"
    
    matchup_outcome_user_input = input (prompt_text) # show prompt and collect user input
    valid_input="no"
    while valid_input=="no":
        user_input = matchup_outcome_user_input    
        if(matchup_outcome_user_input=="1" or matchup_outcome_user_input=="2" or matchup_outcome_user_input=="3"):
            valid_input="yes"
            if(matchup_outcome_user_input=="1"):
                output_msg = current_matchup[0]['name'] + " wins"
                data_output = [1,0]
            elif(matchup_outcome_user_input=="2"):
                output_msg = current_matchup[1]['name'] + " wins"
                data_output = [0,1]
            elif(matchup_outcome_user_input=="3"):
                output_msg = "Tie"
                data_output = [.5,.5]
            print(output_msg)
        else:
            matchup_outcome_user_input = input (prompt_text_err) # show prompt and collect user input
    return(data_output)

In [15]:
# convert outcome data [e.g., 0, 1] into new elo scores after match
def get_new_elo_scores(matchup_outcome):    
    if(matchup_outcome[0]==0):
        blue_new_elo = elo_prediction_data['Adj_Rating_if_Red_Wins'][0]
        red_new_elo = elo_prediction_data['Adj_Rating_if_Red_Wins'][1]
        outcome_text = "L | W"
        wlt = [0,1,0],[1,0,0] # blue: win/loss/tie; red: w/l/t (used as additive points to columns)        
    elif(matchup_outcome[0]==1):
        blue_new_elo = elo_prediction_data['Adj_Rating_if_Blue_Wins'][0]
        red_new_elo = elo_prediction_data['Adj_Rating_if_Blue_Wins'][1]
        outcome_text = "W | L"
        wlt = [1,0,0],[0,1,0] # blue: win/loss/tie; red: w/l/t (used as additive points to columns)
    elif(matchup_outcome[0]==.5):
        blue_new_elo = elo_prediction_data['Adj_Rating_if_Tie'][0]
        red_new_elo = elo_prediction_data['Adj_Rating_if_Tie'][1]
        outcome_text = "T | T"
        wlt = [0,0,1],[0,0,1] # blue: win/loss/tie; red: w/l/t (used as additive points to columns)
    return(blue_new_elo, red_new_elo, outcome_text, wlt)    

In [16]:
# create log of match history

## first make empty dataframe to append rows to
matchup_history = pd.DataFrame()

## append columns to an empty dataframe
matchup_history['Seq'] = []
matchup_history['Matchup'] = []
matchup_history['elo_before_match'] = []
matchup_history['Exp_Win_Pct'] = []
matchup_history['Prediction'] = []
matchup_history['Outcome'] = []
matchup_history['elo_after_match'] = []

In [17]:
# update matchup history
def update_matchup_history(matchup_history, current_matchup, elo_prediction_data, new_elo_scores):
    new_row = {
        'Seq': len(matchup_history)+1,
        'Matchup': current_matchup[0]['name'] + " vs " + current_matchup[1]['name'],
        'elo_before_match': str(current_matchup[0]['elo_score']) + " | " + str(current_matchup[1]['elo_score']),
        'Exp_Win_Pct': elo_prediction_data['exp_win_pct'][0] + " | " + elo_prediction_data['exp_win_pct'][1],
        'Prediction': elo_prediction_data['prediction'][0] + " | " + elo_prediction_data['prediction'][1],
        'Outcome': new_elo_scores[2],
        'elo_after_match': new_elo_scores[0].round(2).astype(str) + " | " + new_elo_scores[1].round(2).astype(str) # read variable as string
    }
    matchup_history.loc[len(matchup_history)] = new_row
    return matchup_history

In [18]:
# update data after matchup ends
def update_data():
    # get target names
    blue_name = current_matchup[0]['name']
    red_name = current_matchup[1]['name']

    # update data
    data.loc[(data['name'] == blue_name),'elo_score'] = new_elo_scores[0].round(decimals=2) # update blue's elo
    data.loc[(data['name'] == red_name),'elo_score'] = new_elo_scores[1].round(decimals=2) # update red's elo
    data.loc[(data['name'] == blue_name),'vs_count'] += 1 # update blue's "vs_count"
    data.loc[(data['name'] == red_name),'vs_count'] += 1 # update reds's "vs_count"
    data.loc[(data['name'] == blue_name),'wins'] = data.loc[(data['name'] == blue_name),'wins'] + new_elo_scores[3][0][0] # update blue wins
    data.loc[(data['name'] == blue_name),'losses'] = data.loc[(data['name'] == blue_name),'losses'] + new_elo_scores[3][0][1] # update blue losses 
    data.loc[(data['name'] == blue_name),'ties'] = data.loc[(data['name'] == blue_name),'ties'] + new_elo_scores[3][0][2]# update blue ties
    data.loc[(data['name'] == red_name),'wins'] = data.loc[(data['name'] == red_name),'wins'] + new_elo_scores[3][1][0] # update red wins    
    data.loc[(data['name'] == red_name),'losses'] = data.loc[(data['name'] == red_name),'losses'] + new_elo_scores[3][1][1] # update red losses    
    data.loc[(data['name'] == red_name),'ties'] = data.loc[(data['name'] == red_name),'ties'] + new_elo_scores[3][1][2] # update red ties
    data.loc[(data['name'] == blue_name),'win_pct'] = data.loc[(data['name'] == blue_name),'wins'] / data.loc[(data['name'] == blue_name),'vs_count']
    data.loc[(data['name'] == red_name),'win_pct'] = data.loc[(data['name'] == red_name),'wins'] / data.loc[(data['name'] == red_name),'vs_count']
    data.loc[(data['name'] == blue_name),'vs_history'] = data.loc[(data['name'] == blue_name),'vs_history'] + [current_matchup[2]] + "; " # add new item to blue's vs_history
    data.loc[(data['name'] == red_name),'vs_history'] = data.loc[(data['name'] == red_name),'vs_history'] + [current_matchup[2]] + "; "# add new item to red's vs_history
    
    # update formatting
    data.loc[(data['name'] == blue_name),'win_pct'] = data.loc[(data['name'] == blue_name),'win_pct'].astype(float).round(decimals=2)
    data.loc[(data['name'] == red_name),'win_pct'] = data.loc[(data['name'] == red_name),'win_pct'].astype(float).round(decimals=2)
    

In [19]:
# mega function: run dat shit
def run_matchup(data, matchup_history): # uses "data" dataframe    
    global elo_prediction_data, new_elo_scores, matchup_history_record, current_matchup  # make variable known outside function
    current_matchup = matchup_selector(data) # get next matchup
    matchup_outcome = matchup_outcome_user_input() # input matchup outcome       
    elo_prediction_data = elo_prediction(current_matchup[0]['elo_score'], current_matchup[1]['elo_score'])
    new_elo_scores = get_new_elo_scores(matchup_outcome) # get new elo scores from matchup outcome
    matchup_history_record = update_matchup_history(matchup_history, current_matchup, elo_prediction_data, new_elo_scores)
    update_data() # update main dataset with after matchup
    reorder_data() # re-order data

    # return(data, matchup_history_record) # show data and matchup history after a match up

In [20]:
# run tournament function
def run_tournament():
    global data
    max_games = max_game_calc(name_list, 1) # max number of games to be played
    if(data['vs_count'].sum()==0):
        matches_played = 0 # start counter
    else:        
        matches_played = data['vs_count'].sum()/2
    play_match = "yes" # set initial status for loop
    
    while play_match=="yes":
        run_matchup(data, matchup_history) # run matchup
        matches_played +=1
        
        if(matches_played==max_games):
            play_match = "no"
            print("Tournament complete.")
        else:
            prompt_text2 = "\nAnother match? [input: y, n]\n"
            prompt_text2err = "\nInvalid input.\nAnother match? [input: y, n]\n"
            
            valid_input2 = "no"
            while valid_input2 == "no":
                another_matchup_user_input = input(prompt_text2)
                if another_matchup_user_input == "y":
                    valid_input2 = "yes"
                    play_match = "yes"
                elif another_matchup_user_input == "n":
                    valid_input2 = "yes"
                    play_match = "no"
                    print("\n \nMatches played: " + str(matches_played) + "/" + str(max_games))
                else:
                    print("Invalid input.")

In [28]:
run_tournament() # function for starting/continuing tournament

1 vs 4
Who wins? [input: 1, 2, 3] 
1) 1
2) 4
3) Tie
1
1 wins

Another match? [input: y, n]
y
3 vs 2
Who wins? [input: 1, 2, 3] 
1) 3
2) 2
3) Tie
2
2 wins

Another match? [input: y, n]
y
5 vs 1
Who wins? [input: 1, 2, 3] 
1) 5
2) 1
3) Tie
2
1 wins

Another match? [input: y, n]
n

 
Matches played: 8.0/10.0


In [31]:
ranked_data(data) # View ranked data for competitors

,initial_order_number,name,elo_score,vs_count,wins,losses,ties,win_pct,vs_history
0,3,1,1058.97,4,4,0,0,1.0,1 vs 3; 1 vs 2; 1 vs 4; 1 vs 5;
1,4,2,1016.68,3,2,1,0,0.67,2 vs 4; 1 vs 2; 2 vs 3;
3,1,4,984.76,3,1,2,0,0.33,4 vs 5; 2 vs 4; 1 vs 4;
2,2,3,984.03,3,1,2,0,0.33,1 vs 3; 3 vs 5; 2 vs 3;
4,0,5,955.56,3,0,3,0,0.0,4 vs 5; 3 vs 5; 1 vs 5;


In [32]:
matchup_history_record # see history of matchups for tournament

,Seq,Matchup,elo_before_match,Exp_Win_Pct,Prediction,Outcome,elo_after_match
0,1.0,5 vs 4,1000 | 1000,0.50 | 0.50,T | T,L | W,984.0 | 1016.0
1,2.0,3 vs 1,1000 | 1000,0.50 | 0.50,T | T,L | W,984.0 | 1016.0
2,3.0,2 vs 4,1000 | 1016,0.48 | 0.52,L | W,W | L,1016.74 | 999.26
3,4.0,1 vs 2,1016.0 | 1016.74,0.50 | 0.50,L | W,W | L,1032.03 | 1000.71
4,5.0,5 vs 3,984.0 | 984.0,0.50 | 0.50,T | T,L | W,968.0 | 1000.0
5,6.0,1 vs 4,1032.03 | 999.26,0.55 | 0.45,W | L,W | L,1046.53 | 984.76
6,7.0,3 vs 2,1000.0 | 1000.71,0.50 | 0.50,L | W,L | W,984.03 | 1016.68
7,8.0,5 vs 1,968.0 | 1046.53,0.39 | 0.61,L | W,L | W,955.56 | 1058.97
